In [ ]:
import os
from langchain.agents import initialize_agent, AgentType
from langchain_azure_ai.chat_models import AzureAIChatCompletionsModel
from langchain_community.agent_toolkits.github.toolkit import GitHubToolkit
from langchain_community.utilities.github import GitHubAPIWrapper
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from github import Github
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.tools import tool

In [ ]:
endpoint = "https://models.github.ai/inference"
model = "openai/gpt-4.1"
token = os.environ["GH_OPENAI_TOKEN"]

github_repository = "suryaatul1/genai-agentic-github-readme"


github1 = GitHubAPIWrapper()
toolkit = GitHubToolkit.from_github_api_wrapper(github1)

tools = toolkit.get_tools()

for tool in tools:

    print(tool.name)


In [ ]:
llm = AzureAIChatCompletionsModel(

                endpoint="https://models.github.ai/inference",
                credential=os.environ['GH_OPENAI_TOKEN'],
                #model="openai/gpt-4.1",
                #model="mistral-ai/Mistral-Large-2411",
                model="openai/gpt-4.1-mini",
                temperature=0,
                top_p=0.9
)

In [ ]:
@tool
def read_markdown_file(file_path: str) -> str:
    """
    Reads and returns the content of a markdown (.md) file from a GitHub repository.
    """
    if not file_path.endswith('.md'):
        return "Only markdown (.md) files are supported."
    try:
        g = Github(os.environ["GH_OPENAI_TOKEN"])
        repo = g.get_repo("suryaatul1/genai-agentic-github-readme")
        file_content = repo.get_contents(file_path)
        return file_content.decoded_content.decode()
    except Exception as e:
        return f"Error reading file: {e}"

In [ ]:
# Define the system message with a variable
system_template = "You are a helpful assistant in searching and reading the markdown format files with .md extensions for given {topic}."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

# Define the human message with a variable
human_template = "Please help me find the .md file and its path in the github repository  for Topic :{topic} . it should be in the format of 'path/to/file.md'."
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

human_template_read = "The path returned should be used to read the .md file for the Topic :{topic} ."
human_message_prompt_read = HumanMessagePromptTemplate.from_template(human_template)

human_template1 = "Please help me summarize the content of the file and highlight key features for Topic :{topic} so a 5th grader can understand it."
human_message_prompt1 = HumanMessagePromptTemplate.from_template(human_template1)

# Combine them into a ChatPromptTemplate
chat_prompt_template = ChatPromptTemplate.from_messages([
    system_message_prompt,
    human_message_prompt,
    human_message_prompt_read,
    human_message_prompt1
])


prompt_arguments = {
    "topic": "Spring Boot . Extract the documentation from content."
}

# prompt_arguments = {
#      "topic": "what is spring boot used for."
#  }

# prompt_arguments = {
#      "topic": "list me all the markdowm files."
#  }

formatted_prompt = chat_prompt_template.format_prompt(**prompt_arguments)

tools.append(read_markdown_file)

# Initialize the agent
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)


# Ask the agent to read a markdown file
result = agent.invoke(formatted_prompt)

print(result)

